<hr/>

<b>Notebook Summary</b>

These notes are based on Prof. Norman Wildberger's lectures on Dynamics on Graphs which can be found <a href="https://www.youtube.com/c/WildEggmathematicscourses/featured">here</a>. 
    
They notes are are being hosted at my website <a href="https://www.ladatavita.com/">ladatavita.com</a> and the Jupyter notebook is also available from my Github repo at: <a href="https://github.com/jgab3103/Jamie-Gabriel/tree/main/MathNotebooks">https://github.com/jgab3103/Jamie-Gabriel/tree/main/MathNotebooks</a>

The purpose of this notebook is a continuation where we left off in the previous notebook ES2, which compared the root populations of ADE graphs and ADE~ graph. 

This notebook will continue the investigation.... 

<hr/>

In [2]:
import pyvis.network as nt
import numpy as np
import sympy as sp
from IPython.display import HTML
import ipywidgets as widgets
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
mpl.rcParams['legend.fontsize'] = 10
import pandas as pd
import networkx as nx
import string
import random

<b>Let</b> $F1, F2$ and $F3$ be functions (created in the previous notebook) to create graphs given connections of edges, carry out the $ps_x$ function, and test this function over a chosen amount of iterations

In [3]:
def F1(edges=None):
    
    verticeLabels = list(set([item for sublist in edges for item in sublist]))
    randomNumberToIntialiseSingletonPopulation = random.randint(0, len(verticeLabels) - 1)

    vertices = [(verticeLabels[i], {"population": 0}) for i in range(len(verticeLabels))]
    graph = nx.Graph()
    graph.add_nodes_from(vertices)
    graph.add_edges_from(edges)
    
    graph.nodes[verticeLabels[randomNumberToIntialiseSingletonPopulation]]['population'] = 1
    
    return(graph)

def F2(graph = None, nodeChoice = None, printSummary = True, 
       returnUpdatedGraph = False, 
       returnListOfPopulations = False,
      returnAllAsDict = False):
   
    edgesOfChosenNode = list(nx.edges(graph, [nodeChoice]))
   
    neigborOfChosenNode = [edgesOfChosenNode[i][1] for i in range(len(list(edgesOfChosenNode)))]
    nodeChoicePopulation = graph.nodes[nodeChoice]['population']
    sumOfNeighborsOfChosenNode = np.sum([graph.nodes[i]['population'] for i in neigborOfChosenNode])
    populationOfNode = -nodeChoicePopulation + sumOfNeighborsOfChosenNode
    updatedGraph = graph.copy()
    updatedGraph.nodes[nodeChoice]['population'] = populationOfNode

    newPopulations = [updatedGraph.nodes[i]['population'] for i in list(updatedGraph)]
    if printSummary:
        print("Node choice", 
              nodeChoice,
              "\nNode details",
              nx.nodes(graph)[nodeChoice],
              "\nChange in node population ",
              nx.nodes(graph)[nodeChoice]['population'], 
              "->", 
              populationOfNode)
        print("Updated node populations of graph: ", newPopulations, "\n")

    if returnUpdatedGraph: 
        return(updatedGraph)
    
    if returnListOfPopulations:
        return(np.array(newPopulations))
    
    if returnAllAsDict:
        return({"graph": updatedGraph,
               "population": np.array(newPopulations)})
    
def F3(graphChoice=None, vertices = [], iterations=5, returnPopulations = True, iterateThroughAllVertices = True):
    listOfAllPopulations = []
    graph = graphChoice
    
    if iterateThroughAllVertices:
        vertices = graph.nodes
    
    for i in range(iterations):
        for j in vertices:
            udpatedGraphAndPopulation = F2(graph, j, returnAllAsDict=True, printSummary = False)
            graph = udpatedGraphAndPopulation['graph']
            population = udpatedGraphAndPopulation['population']
            listOfAllPopulations.append(population)
            
    print("Number of unique populations: ", len(list(set([tuple(i) for i in [list(i) for i in listOfAllPopulations]]))))
    if returnPopulations:
        setOfAllPopulations = list(set([tuple(i) for i in [list(i) for i in listOfAllPopulations]]))
        return({"iterations":iterations,
               "populationAsSetCount":len(setOfAllPopulations),
                "populationAsListCount":len(listOfAllPopulations),
               "populationsAsSet": setOfAllPopulations,
               "populationsAsList": listOfAllPopulations})

<b>Recall</b> the usage of these functions with the following example.

<b>Let</b> $F4$ be an example of a graph created with the $F1$ function

In [4]:
F4 = F1(edges=[("a","b"),("b","c"),("c","d"),("d","e"),("e","f")] )

<b>Let</b> $F5$ be an example of a function that applies the $ps_x$ mutation to a given graph and vertice.

In [5]:
F5 = F2(graph=F4, nodeChoice='a')

Node choice a 
Node details {'population': 0} 
Change in node population  0 -> 0
Updated node populations of graph:  [0, 0, 0, 1, 0, 0] 



<b>Let</b> $F6$ be an example of a function that applies the $ps_x$ mutation to a chosen set of nodes over  chosen set of iterations. 

In [6]:
F6 = F3(graphChoice=F4, iterations=10, returnPopulations=True)

Number of unique populations:  22


<hr/>

<b>Aim</b> Explore the ADE~ graphs in more detail to understand their importance and independent of interest, and realtion to stagle population

<hr/>

<b>Let</b> $X$ be a connected simple graph. 

<b>Let</b> $P(X) \equiv \text{populations on } X$.

<b>Recall</b> that for a the vertex, $x$ in the graph $X$, the population function $S_x:P(x)  \rightarrow P(X)$ is defined by: 

$$ ps_x(y) \equiv  \begin{cases}
    -p(x) + \Sigma_{z \in N(x)} \text{ }  p(z) & \text{if y = x}\\
    p(y) & \text{otherwise}
\end{cases}
$$ 

<b>Definition</b>: A population, $p \in P(X)$ is <b>stable</b> $\iff$ $ps_x = p$ for any vertex $x$ of a connected simple graph $X$, meaning that it is left invariant under every mutation and that population will not change when the mutation function is applied.

<b>Observe</b> that this is immedieately equivalent to the condition: 

$$ 2p(x) = \Sigma_{z\in N(x)} p(z) $$

for any vertex $x$ of $X$ or

$$ p(x) = \frac{1}{2} \Sigma_{z \in N(x)} p(z) $$